In [1]:
#import all the modeules we've used before

import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import rdkit
# Render the figure in a notebook:
%matplotlib inline  

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
Nan0_xtest = pd.read_csv('X_testNan1scaled_Desc.csv', index_col=0)
Nan0_xtrain = pd.read_csv('X_trainNan1scaled_Desc.csv', index_col=0)
Nan0_xval = pd.read_csv('X_valNan1scaled_Desc.csv', index_col=0)

In [3]:

Nan0_ytest = pd.read_csv('y_testNan1.csv', index_col=0)
Nan0_ytrain = pd.read_csv('y_trainNan1.csv', index_col=0)
Nan0_yval = pd.read_csv('y_valNan1.csv', index_col=0)

In [4]:
num_rows_with_nan = Nan0_xtrain.isna().any(axis=1).sum()
print(num_rows_with_nan)
### Nan exist in desc. dataframe, need to be removed

100


In [5]:
# Nan0_xtest.isna().stack()[lambda x: x].index
# Nan0_xtest.index[Nan0_xtest.isna().any(axis=1)]

# rows, cols = np.where(Nan0_xtest.isna())

# for r, c in zip(rows, cols):
#     print(f"NaN at row={Nan0_xtest.index[r]}, column={Nan0_xtest.columns[c]}")



In [6]:
mask = Nan0_xtrain.notna().all(axis=1)

Nan0_xtrain_clean = Nan0_xtrain[mask]
Nan0_ytrain_clean = Nan0_ytrain[mask]

mask2 = Nan0_xtest.notna().all(axis=1)

Nan0_xtest_clean = Nan0_xtest[mask2]
Nan0_ytest_clean = Nan0_ytest[mask2]

mask3 = Nan0_xval.notna().all(axis=1)

Nan0_xval_clean = Nan0_xval[mask3]
Nan0_yval_clean = Nan0_yval[mask3]

In [7]:
num_rows_with_nan_clean = Nan0_xtrain_clean.isna().any(axis=1).sum()
print(num_rows_with_nan_clean)

0


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn import metrics
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn import metrics

In [10]:
###Define a XGBC Model for Optimisation
XGBC = XGBClassifier(
    eta=0.3,
    gamma=0,
    max_depth=6,
    subsample=1,
    min_child_weight=1,
    objective='binary:logistic',   # binary classification
    eval_metric='logloss'          # or 'auc'
)

XGBC.fit (Nan0_xtrain_clean, Nan0_ytrain_clean)
y_pred = XGBC.predict(Nan0_xtest_clean)




In [11]:

#Now optimise it using GridSearch
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    "n_estimators": [100, 300],
    "max_depth": [3, 6],
    "learning_rate": [0.1, 0.3]
}

# Grid search with 5-fold CV
grid = GridSearchCV(XGBC, param_grid, cv=5, scoring="balanced_accuracy")
grid.fit(Nan0_xtrain_clean, Nan0_ytrain_clean)

# Best parameters and CV score
print("Best params:", grid.best_params_)
print("Best balanced accuracy:", grid.best_score_)

# Best params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
# Best CV accuracy: 0.7171948233076011




Best params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100}
Best balanced accuracy: 0.6531999512262697


In [14]:
#Now run the pipeline with the optimised model#######################16/12/2025 Ends Here!!!
XGBC_op = XGBClassifier(
    n_estimators = 100,
    eta=0.1,
    gamma=0,
    max_depth=6,
    subsample=1,
    min_child_weight=1,
    objective='binary:logistic',   # binary classification
    eval_metric='logloss'          # or 'auc'
)

XGBC_op.fit (Nan0_xtrain_clean, Nan0_ytrain_clean)
y_pred_op = XGBC_op.predict(Nan0_xtest_clean)

print("Accuracy:", balanced_accuracy_score(Nan0_ytest_clean, y_pred_op))
# print("\nConfusion Matrix:\n", confusion_matrix(Nan0_ytest_clean, y_pred))

Accuracy: 0.6468411996066863


In [16]:
from sklearn import metrics
def get_model_metrics(fitted_model, x_train, x_val, y_train, y_val):
    pred_X_train = fitted_model.predict(x_train)
    pred_X_val = fitted_model.predict(x_val)
    train_bal_acc = metrics.balanced_accuracy_score(y_train, pred_X_train)
    val_bal_acc = metrics.balanced_accuracy_score(y_val, pred_X_val)
    train_recall = metrics.recall_score(y_train, pred_X_train)
    val_recall = metrics.recall_score(y_val, pred_X_val)
    train_precision = metrics.precision_score(y_train, pred_X_train)
    val_precision = metrics.precision_score(y_val, pred_X_val)
    train_f1 = metrics.f1_score(y_train, pred_X_train)
    val_f1 = metrics.f1_score(y_val, pred_X_val)
    metric_list = [train_bal_acc, train_recall, train_precision, train_f1, val_bal_acc, val_recall, val_precision, val_f1]
    print(metric_list)

get_model_metrics(XGBC_op, Nan0_xtrain_clean, Nan0_xval_clean, Nan0_ytrain_clean, Nan0_yval_clean )

[0.9200297776299224, 0.9635388739946381, 0.9465367395312089, 0.9549621363092866, 0.637195292017627, 0.8540609137055838, 0.7613122171945701, 0.8050239234449761]


In [15]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.inspection import permutation_importance
from sklearn.metrics import balanced_accuracy_score

baseline = balanced_accuracy_score(
    Nan0_ytest_clean,
    XGBC_op.predict(Nan0_xtest_clean) #calculates a balanced accuracy score using the optimised model
)

importances = [] #empty list generation

for col in Nan0_xtest_clean.columns:
    X_perm = Nan0_xtest_clean.copy() #makes a copy of the df
    X_perm[col] = np.random.permutation(X_perm[col]) #goes through the columns in the copied df and shuffles the features

    score = balanced_accuracy_score(
        Nan0_ytest_clean,
        XGBC_op.predict(X_perm)
    ) #calculates a balanced accuracy score with shuffled features

    importances.append(baseline - score) #subtracts the scores the larger the performance drop and the output the more important the feature

importances = pd.DataFrame(importances)

importances

importances_df = pd.DataFrame({
    "feature": Nan0_xtest_clean.columns,
    "importance": importances[0]   # your single-column DataFrame
})

# Get the top 25 features
fr_features = importances_df[importances_df["feature"].str.startswith("fr_")]

# Get top 25
top25_fr = fr_features.nlargest(25, "importance").reset_index(drop=True)

top25_fr

,feature,importance
0,fr_phenol_noOrthoHbond,0.004167
1,fr_alkyl_halide,0.003535
2,fr_benzene,0.002778
3,fr_phenol,0.001514
4,fr_NH0,0.001389
5,fr_aniline,0.001389
6,fr_bicyclic,0.001389
7,fr_ether,0.001389
8,fr_Al_COO,0.000000
9,fr_Al_OH_noTert,0.000000
